## Assignment 3 - Nick Jones (nj935@nyu.edu)

### What strategies work for the hard-to-reach.
Reproducing the results on likelihood of re-offending and being convicted of a felony after
taking part in the program.

In [236]:
import os
import sys
import numpy as np
import pylab as pl
import pandas as pd

import json
import os

%pylab inline

'''Question: are participants in the program less likely to re-offend and be
convicted of a felony than non-participants?

Data:
 - conviction rates after three years for the control group (P0)
 - conviction rates after three years for test group (P1)

Null hypothesis
H0: P0=<P1
H1: P0>P1

significance level p = 0.05'''

alpha=0.05
P_0 = 11.7 * 0.01 
P_1 = 10.0 * 0.01

if P_0 - P_1 <= 0:
    print ("the Null holds")
else:
    print ("we must assess the statistical significance")

# get counts of re-offenders by multiplying p * sample size
n_0 = 409 
n_1 = 564 

Nt_0 = P_0 * n_0
Nt_1 = P_1 * n_1

# functions for sample proportions and standard error
p = lambda p0, p1, n0, n1: (p0 * n0 + p1 * n1) / (n0 + n1)
se = lambda p, n0, n1: np.sqrt(p * (1 - p) * (1.0 / n0 + 1.0 / n1))

# Z-score: how many standard deviations is the sample from the population 
zscore = lambda p0, p1, s : (p0 - p1) / s
z_3y = zscore(P_1, P_0, se(p(P_0, P_1, n_0, n_1), n_0, n_1))
print ("Z score: %.4f"%z_3y)

''' Generates z score: -0.8463
 from look-up table, the probability of getting the
 z-score -0.846283 is 0.7995'''
 
p_3y = 1 - 0.7995

def report_result(p,a):
    print ('is the p value ' + 
           '{0:.4f} smaller than the critical value {1:.2f}?'.format(p,a))
    if p < a:
        print ("YES!")
    else: 
        print ("\nNO!")
    
    print ('the Null hypothesis is {}'.format(\
                            'rejected' if p < a  else 'not rejected') )

report_result(p_3y, alpha)

Populating the interactive namespace from numpy and matplotlib
we must assess the statistical significance
Z score: -0.8463


TypeError: float argument required, not function

### Look at recidivism using chi-square test instead

In [198]:
def evalChisq(values):
    '''Evaluates the chi sq from a contingency value
    Arguments:
    values: 2x2 array or list, the contingengy table
    '''
    if not (len(values.shape) == 2 and values.shape == (2,2)):
        print ("must pass a 2D array")
        return -1
    values = np.array(values)
    E = np.empty_like(values)
    for j in range(len(values[0])):
        for i in range(2):
            
            E[i][j] = ((values[i,:].sum() * values[:,j].sum()) / 
                        (values).sum())
    return ((values - E)**2 / E).sum()

In [194]:
# Build contingency table

rec_table = np.zeros(shape=(3,3))
rt = pd.DataFrame(data=rec_table, columns=("yes","no","total"), index=('test','control','total'))

test_N = [0.1*568,(1-0.1)*568]   # prob times values for the test group
test_N = np.array(test_N)
test_N = np.append(test_N,test_N.sum())   # add in the total

control_N = [0.117*409,(1-0.117)*409]   # and for the control group
control_N = np.array(control_N)
control_N = np.append(control_N,control_N.sum())   # add in the total

rt.iloc[0,] = test_N
rt.iloc[1,] = control_N

convicted = rt.yes.sum()
not_convicted = rt.no.sum()
grand_total = rt.total.sum()

rt.iloc[2,0] = convicted
rt.iloc[2,1] = not_convicted
rt.iloc[2,2]= grand_total
rt


,yes,no,total
test,56.800,511.200,568.0
control,47.853,361.147,409.0
total,104.653,872.347,977.0


In [224]:
# Chi-squared test at p=0.05

test_values=np.vstack((test_N[0:2],control_N[0:2]))

crit_value = 3.84

test_stat = evalChisq(test_values)

print("Critical value: %s \nTest statistic: %s" % (crit_value,test_stat))
if test_stat <= crit_value:
    print("\nNull hypothesis is not rejected!")
else:
    print("Null hypothesis rejected")
    

Critical value: 3.84 
Test statistic: 0.718493917505

Null hypothesis is not rejected!
